In [3]:
import string
import secrets
from passlib.context import CryptContext

class PasswordManager:
    def __init__(self):  # Corrected to __init__
        """Initialize password manager with history and cryptographic context."""
        self.history = []
        self.crypt_context = CryptContext(schemes=["bcrypt"], deprecated="auto")
    
    def encrypt_password(self, password):
        """Encrypt the password with a generated salt."""
        salt = secrets.token_hex(16)
        return self.crypt_context.hash(password + salt), salt

    def check_password_strength(self, password):
        """Assess password strength based on entropy."""
        length = len(password)
        char_types = sum(
            bool(set(chars) & set(password)) for chars in [string.ascii_lowercase, string.ascii_uppercase, string.digits, string.punctuation]
        )
        entropy = length * char_types
        if entropy < 40:
            return "Weak"
        elif entropy < 60:
            return "Moderate"
        else:
            return "Strong"

    def generate_password(self, length=16, use_lower=True, use_upper=True, use_digits=True, use_symbols=True):
        """Generate a password based on selected character types."""
        lower = string.ascii_lowercase if use_lower else ""
        upper = string.ascii_uppercase if use_upper else ""
        digits = string.digits if use_digits else ""
        symbols = string.punctuation if use_symbols else ""
        
        char_set = lower + upper + digits + symbols
        if not char_set:
            raise ValueError("At least one character type must be selected.")
        
        password = [
            secrets.choice(lower) if use_lower else '',
            secrets.choice(upper) if use_upper else '',
            secrets.choice(digits) if use_digits else '',
            secrets.choice(symbols) if use_symbols else ''
        ]
        
        password += [secrets.choice(char_set) for _ in range(length - len(password))]
        
        secrets.SystemRandom().shuffle(password)
        
        password = ''.join(password)

        encrypted_password, salt = self.encrypt_password(password)
        self.history.append((password, encrypted_password, salt))
        return password, encrypted_password, salt

    def display_password_best_practices(self):
        """Display password best practices for users."""
        print("Password Best Practices:")
        print("- Minimum length of 12 characters.")
        print("- Use a mix of upper and lowercase letters, numbers, and symbols.")
        print("- Avoid common patterns or sequences.")
        print("- Do not reuse passwords.")

    def check_common_patterns(self, password):
        """Check for common weak patterns in the password."""
        common_patterns = ["123", "password", "admin", "qwerty", "abc"]
        return any(pattern in password for pattern in common_patterns)

    def display_password_history(self):
        """Display the history of generated passwords (plain and encrypted)."""
        print("Password History:")
        for i, (plain, encrypted, salt) in enumerate(self.history, 1):
            print(f"{i}. Plain: {plain}, Encrypted: {encrypted}, Salt: {salt}")

# Create password manager instance
password_manager = PasswordManager()

# User input section
try:
    length = int(input("Enter password length (12-64): "))
    use_lower = input("Include lowercase letters? (y/n): ").lower() == 'y'
    use_upper = input("Include uppercase letters? (y/n): ").lower() == 'y'
    use_digits = input("Include digits? (y/n): ").lower() == 'y'
    use_symbols = input("Include symbols? (y/n): ").lower() == 'y'

    # Generate password
    password, encrypted_password, salt = password_manager.generate_password(
        length=length,
        use_lower=use_lower,
        use_upper=use_upper,
        use_digits=use_digits,
        use_symbols=use_symbols
    )

    # Display password
    print("\nGenerated Password:", password)
    print("Encrypted Password:", encrypted_password)
    print("Salt:", salt)

    # Check password strength
    strength = password_manager.check_password_strength(password)
    print("Password Strength:", strength)

    # Check common patterns
    if password_manager.check_common_patterns(password):
        print("Warning: The password contains common patterns, consider regenerating.")

    # Display password best practices
    password_manager.display_password_best_practices()

    # Display password history
    password_manager.display_password_history()

except ValueError:
    print("Invalid input. Please enter a valid number for password length.")



Generated Password: BBOHnCX'rCV&e.~G~
Encrypted Password: $2b$12$UnX6y/qg7A2XBw/QLQf7Ke0zvrwhdJmsvaTuo2eDPKKRJDZrfmjM6
Salt: ca5b9529416c9f76581d3f960d556ef6
Password Strength: Moderate
Password Best Practices:
- Minimum length of 12 characters.
- Use a mix of upper and lowercase letters, numbers, and symbols.
- Avoid common patterns or sequences.
- Do not reuse passwords.
Password History:
1. Plain: BBOHnCX'rCV&e.~G~, Encrypted: $2b$12$UnX6y/qg7A2XBw/QLQf7Ke0zvrwhdJmsvaTuo2eDPKKRJDZrfmjM6, Salt: ca5b9529416c9f76581d3f960d556ef6
